In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Options d'affichage
pd.set_option('display.max_columns', None)

# Chargement ultra-rapide
df = pd.read_parquet("sas_sarl_filtered.parquet")

# Vérification rapide
print(f"Dimensions du dataset : {df.shape}")
df.head()

In [ ]:
df.info(verbose=True)

In [ ]:
print("Number of rows : {}".format(df.shape[0]))
print()

print("Basics statistics: ")
df_desc = df.describe(include="all")
display(df_desc)
print()

---

In [ ]:
df.shape

#### Traitement des valeurs nulles

In [ ]:
# Calculer le pourcentage de NaN par colonne
nan_percent = df.isna().mean() * 100

# Sélectionner seulement les colonnes avec au moins un NaN
nan_columns = nan_percent[nan_percent > 0].sort_values(ascending=False)

# Afficher
print(nan_columns)

#### Suppression des colonnes inutiles

In [ ]:
# Liste des colonnes à supprimer
colonnes_a_drop = [
    "Libellé cedex de l'établissement",
    "Numero de voie de l'établissement 2",
    "Distribution spéciale de l'établissement",
    "Libellé du pays de l'établissement étranger.1",
    "Prénom usuel de la personne physique",
    "Prénom de la personne physique 3",
    "Indice de répétition de l'établissement 2",
    "Complément d'adresse de l'établissement 2",
    "Code cedex de l'établissement",
    "Code pays de l'établissement",
    "Libellé cedex de l'établissement 2",
    "Code cedex de l'établissement 2",
    "Code de la commune de l'établissement 2",
    "Distribution spéciale de l'établissement 2",
    "Libellé de la commune de l'établissement étranger 2",
    "Libellé de la commune de l'établissement 2",
    "Code postal de l'établissement 2",
    "Libellé de la voie de l'établissement 2",
    "Type de voie de l'établissement 2",
    "Pseudonyme de la personne physique",
    "Prénom de la personne physique 1",
    "Civilité de la personne physique",
    "Prénom de la personne physique 4",
    "Caractère employeur de l'unité légale",
    "Première ligne de l'adressage",
    "Nom de la personne physique",
    "Nom d'usage de la personne physique",
    "Prénom de la personne physique 2",
    "Unité légale purgée",
    "Libellé de la commune de l'établissement à l'étranger",
    "Code du pays de l'établissement étranger",
    "Identifiant association de l'unité légale",
    "Libellé du pays de l'établissement étranger",
    "Dénomination usuelle de l'unité légale 3",
    "Enseigne de l'établissement 3",
    "Dénomination usuelle de l'unité légale 2",
    "Enseigne de l'établissement 2",
    "Société à mission unité légale",
    "Indice de répétition de l'établissement",
    "Dénomination usuelle de l'unité légale 1",
    "Sigle de l'unité légale",
    "Enseigne de l'établissement 1",
    "Complément d'adresse de l'établissement",
    "Année de la tranche d'effectif de l'établissement",
    "Dénomination usuelle de l'établissement",
    "Activité principale de l'établissement",
    "Année de la tranche de l'effectif de l'unité légale",
    "Etablissement siège",
    "Statut de diffusion de l'établissement",
    "Statut de diffusion de l'unité légale",
    "Tranche de l'effectif de l'établissement",
    "Tranche de l'effectif de l'établissement triable",
    "Numéro de voie de l'établissement",
    "Type de voie de l'établissement",
    "Libellé de la voie de l'établissement",
    
]

# Supprimer les colonnes
df = df.drop(columns=colonnes_a_drop)

In [ ]:
df.shape

In [ ]:
# Calculer le pourcentage de NaN par colonne
nan_percent = df.isna().mean() * 100

# Sélectionner seulement les colonnes avec au moins un NaN
nan_columns = nan_percent[nan_percent > 0].sort_values(ascending=False)

# Afficher
print(nan_columns)

---

#### Traitement des Nan restants

##### 1. ESS

In [ ]:
df['Economie sociale et solidaire unité légale'].value_counts()

In [ ]:
# Remplir les NaN par 'N' (Non) après vérification sur Pappers

df['Economie sociale et solidaire unité légale'] = df['Economie sociale et solidaire unité légale'].replace('', 'N')

print("Vérification finale ESS :")
print(df['Economie sociale et solidaire unité légale'].value_counts())

In [ ]:
# On remplace toutes les chaînes vides par des vrais NaN
df = df.replace('', np.nan)

print(df.isna().mean() * 100)

##### 2. Les dates de fermeture

In [ ]:
df["Date de fermeture de l'unité légale"] = pd.to_datetime(
    df["Date de fermeture de l'unité légale"], errors="coerce"
)
df["Date de fermeture de l'établissement"] = pd.to_datetime(
    df["Date de fermeture de l'établissement"], errors="coerce"
)

In [ ]:
# 🔹 Filtrer uniquement pour l'affichage, sans toucher au df original
df_dates_valide = df[
    df["Date de fermeture de l'unité légale"].notna() &
    df["Date de fermeture de l'établissement"].notna()
].copy()

# 🔹 Ajouter la colonne de correspondance
df_dates_valide["fermeture_correspond"] = (
    df_dates_valide["Date de fermeture de l'unité légale"] ==
    df_dates_valide["Date de fermeture de l'établissement"]
)

# 🔹 Afficher un aperçu
df_dates_valide[["Date de fermeture de l'unité légale",
                 "Date de fermeture de l'établissement",
                 "fermeture_correspond"]].head(10)

In [ ]:
# 🔹 Calculer la part de True / False
part_fermeture = df_dates_valide["fermeture_correspond"].value_counts(normalize=True) * 100

# 🔹 Afficher
print(part_fermeture)

In [ ]:
# 🔹 Créer une colonne "Date_fermeture_finale" avec la date la plus tardive des 2
df["Date_fermeture_finale"] = df[["Date de fermeture de l'établissement", 
                                  "Date de fermeture de l'unité légale"]].max(axis=1)


In [ ]:
df[["Date de fermeture de l'établissement",
    "Date de fermeture de l'unité légale",
    "Date_fermeture_finale"]].head(10)


In [ ]:
# 🔹 Taux de NaN dans la colonne finale
taux_nan = df["Date_fermeture_finale"].isna().mean() * 100

print(f"Taux de NaN dans Date_fermeture_finale : {taux_nan:.2f}%")

##### Application de cette colonne au dataset

In [ ]:
# 🔹 Créer la colonne finale
df["Date_fermeture_finale"] = df[["Date de fermeture de l'établissement",
                                  "Date de fermeture de l'unité légale"]].max(axis=1)

# 🔹 Supprimer les deux anciennes colonnes
df.drop(columns=["Date de fermeture de l'établissement",
                 "Date de fermeture de l'unité légale"], inplace=True)


In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
# Nombre total de NaN dans la colonne finale
nb_nan = df["Date_fermeture_finale"].isna().sum()

print(f"Nombre de NaN dans Date_fermeture_finale : {nb_nan}")

In [ ]:
# 🔹 Afficher les lignes où Date_fermeture_finale est NaN
df_fermeture_manquante = df[df["Date_fermeture_finale"].isna()]

# Affichage des 10 premières lignes pour vérification
df_fermeture_manquante.head(5)


In [ ]:
# 🔹 Supprimer les lignes où Date_fermeture_finale est NaN
df = df.dropna(subset=["Date_fermeture_finale"])

# Vérification
print(f"Nombre de lignes après suppression : {len(df)}")


---

##### Nan restants

In [ ]:
# Calculer le pourcentage de NaN par colonne
nan_percent = df.isna().mean() * 100

# Sélectionner seulement les colonnes avec au moins un NaN
nan_columns = nan_percent[nan_percent > 0].sort_values(ascending=False)

# Afficher
print(nan_columns)

---

In [ ]:
# Suppression des colonnes avec des Nan résiduels
cols_nan = [

    "Géolocalisation de l'établissement",
    "Code EPCI de l'établissement",
    "Libellé de l'EPCI de l'établissement",
    "Code de l'arrondissement de l'établissement",
    "Adresse de l'établissement",
    "Département de l'établissement",
    "Région de l'établissement",
    "Code du département de l'établissement",
    "Code de la région de l'établissement",
    "Code postal de l'établissement",
    "Code commune de l'établissement",
    "Commune de l'établissement",
    "Caractère employeur de l'établissement",
    "Dénomination de l'unité légale"
]

df_clean = df.dropna(subset=cols_nan)

In [ ]:
df_clean.isna().mean().sort_values(ascending=False)


In [ ]:
print("Avant :", df.shape)
print("Après :", df_clean.shape)


### Fin de traitement des Nan

---

#### Vérification des colonnes en doublon

In [ ]:
df_clean.head(5)

---

In [ ]:
correspondance = df_clean["NIC"] == df_clean["NIC du siège de l'unité légale"]
correspondance.sum()

In [ ]:
df_clean.drop(columns=["NIC", "NIC du siège de l'unité légale"], inplace=True)
df_clean = df_clean.reset_index(drop=True)
df_clean.shape

In [ ]:
df_clean.head(5)

#### Traitement des colonnes inutiles

In [ ]:
df_clean = df_clean.drop(columns=["Tranche de l'effectif de l'unité légale triable", 
                      "Catégorie de l'entreprise", 
                      "Année de la catégorie de l'entreprise", 
                      "Classe de l'établissement",
                      "Section de l'unité légale",
                      "Sous-section de l'unité légale",
                      "Division de l'unité légale",
                      "Groupe de l'unité légale",
                      "Classe de l'unité légale",
                      "Nombre de periodes de l'établissement",
                      "Nomenclature de l'activité principale de l'unité légale",
                      "Nomenclature principale de l'établissement",
                      "Code de l'arrondissement de l'établissement",])
df_clean.head()

---

In [ ]:
df_clean.shape

#### Traitement de l'état actif ou cessé

In [ ]:
df_clean['Etat administratif de l\'établissement'].value_counts()

In [ ]:
df_clean['Etat administratif de l\'unité légale'].value_counts()

In [ ]:
df_clean.drop(columns=["Etat administratif de l'établissement"], inplace=True)
df_clean.shape

In [ ]:
df_clean[df_clean["Etat administratif de l'unité légale"] == "Active"].head()


#### Après vérification, les structures sont bien inactives. On drope.


In [ ]:
df_clean = df_clean.drop(columns=["Etat administratif de l'unité légale"])


In [ ]:
df_clean.head(5)

In [ ]:
df_clean.shape

In [ ]:
df_clean.columns.to_list()

---

#### Traitement des SIREN et SIRET sièges

In [ ]:
# Vérification de l'unicité des SIREN et SIRET
df_clean['SIREN'].value_counts().sum()==df_clean['SIRET'].value_counts().sum()

In [ ]:
correspondances = df_clean["SIRET"] == df_clean["SIRET du siège de l'unité légale"]

# Affiche le nombre de True / False
print(correspondances.value_counts())

In [ ]:
df_clean.drop(columns=["SIRET du siège de l'unité légale"], inplace=True)
df_clean.shape

In [ ]:
df_clean.dtypes

In [ ]:
df_clean.head(5)

In [ ]:
# Traitement pour la Région
df_clean["Code de la région de l'établissement"] = (
    df_clean["Code de la région de l'établissement"]
    .astype(str)
    .str.replace('.0', '', regex=False)
)

# Traitement pour l'EPCI
df_clean["Code EPCI de l'établissement"] = (
    df_clean["Code EPCI de l'établissement"]
    .astype(str)
    .str.replace('.0', '', regex=False)
)

In [ ]:
df_clean.dtypes

---

#### Traitement des dates aberrantes

In [ ]:
# Seuil de date raisonnable, ici 2026
seuil_date = '2026-01-23'

# Nombre de lignes avec des dates trop élevées
nb_lignes_aberrantes = df_clean[df_clean['Date_fermeture_finale'] > seuil_date].shape[0]

print(f"Nombre de lignes avec des dates aberrantes : {nb_lignes_aberrantes}")


In [ ]:
# On drop les lignes où la date de fermeture finale dépasse le seuil
df_clean = df_clean[df_clean['Date_fermeture_finale'] <= seuil_date].copy()

# Vérification rapide
print(f"Nouvelle taille du dataset : {df_clean.shape}")


---

#### Export en Parquet

In [ ]:
# df_clean["Date de création de l'unité légale"] = pd.to_datetime(df_clean["Date de création de l'unité légale"], errors='coerce')

# df_clean.to_parquet("dataset_clean.parquet", index=False)